<a href="https://colab.research.google.com/github/dalismo/Finance_News_Sentiment/blob/agonz%2Fworking_train_set/Copy_of_ml_sentiment_analysis_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [445]:
import pandas as pd

import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:5 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:6 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:7 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:8 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:9 http://security.ubuntu.com/ubuntu bionic-security InRelease
Ign:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:13 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64

In [446]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [447]:
#Read additional stop words (Referenced: https://github.com/igorbrigadir/stopwords/blob/master/en/tonybsk_6.txt)
add_stop_words = pd.read_csv('/content/drive/MyDrive/english.txt', header=None)


In [448]:
stop_list=list(add_stop_words[0].astype(str))

In [449]:
len(stop_list)

1298

In [450]:
print(stop_list)

["'ll", "'tis", "'twas", "'ve", '10', '39', 'a', "a's", 'able', 'ableabout', 'about', 'above', 'abroad', 'abst', 'accordance', 'according', 'accordingly', 'across', 'act', 'actually', 'ad', 'added', 'adj', 'adopted', 'ae', 'af', 'affected', 'affecting', 'affects', 'after', 'afterwards', 'ag', 'again', 'against', 'ago', 'ah', 'ahead', 'ai', "ain't", 'aint', 'al', 'all', 'allow', 'allows', 'almost', 'alone', 'along', 'alongside', 'already', 'also', 'although', 'always', 'am', 'amid', 'amidst', 'among', 'amongst', 'amoungst', 'amount', 'an', 'and', 'announce', 'another', 'any', 'anybody', 'anyhow', 'anymore', 'anyone', 'anything', 'anyway', 'anyways', 'anywhere', 'ao', 'apart', 'apparently', 'appear', 'appreciate', 'appropriate', 'approximately', 'aq', 'ar', 'are', 'area', 'areas', 'aren', "aren't", 'arent', 'arise', 'around', 'arpa', 'as', 'aside', 'ask', 'asked', 'asking', 'asks', 'associated', 'at', 'au', 'auth', 'available', 'aw', 'away', 'awfully', 'az', 'b', 'ba', 'back', 'backed', 

In [451]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("StopWords").getOrCreate()
# spark = SparkSession.builder.appName("NaiveBayes").getOrCreate()

In [452]:
from pyspark.ml.feature import Tokenizer, StopWordsRemover
from pyspark.ml.feature import Tokenizer, StopWordsRemover, HashingTF, IDF, StringIndexer


In [453]:
# Read in data from S3 Buckets
from pyspark import SparkFiles

url = 'https://raw.githubusercontent.com/dalismo/Finance_News_Sentiment/main/Resources/train-data.csv'
# df = pd.read_csv(url,encoding='latin-1', header=None)
# url ="https://s3.amazonaws.com/dataviz-curriculum/day_2/yelp_reviews.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("train-data.csv"), sep=",", header=None)

# Show DataFrame
df.show()

+--------+--------------------+
|     _c0|                 _c1|
+--------+--------------------+
| neutral|According to Gran...|
| neutral|Technopolis plans...|
|negative|The international...|
|positive|With the new prod...|
|positive|According to the ...|
|positive|FINANCING OF ASPO...|
|positive|For the last quar...|
|positive|In the third quar...|
|positive|Operating profit ...|
|positive|Operating profit ...|
|positive|TeliaSonera TLSN ...|
|positive|STORA ENSO , NORS...|
|positive|A purchase agreem...|
|positive|Finnish Talentum ...|
|positive|Clothing retail c...|
|positive|Consolidated net ...|
|positive|Foundries divisio...|
|positive|HELSINKI ( AFX ) ...|
|positive|Incap Contract Ma...|
|positive|Its board of dire...|
+--------+--------------------+
only showing top 20 rows



In [454]:
from pyspark.sql.functions import length
# Create a length column to be used as a future feature 
data_df = df.withColumn('length', length(df['_c1']))
data_df.show()

+--------+--------------------+------+
|     _c0|                 _c1|length|
+--------+--------------------+------+
| neutral|According to Gran...|   127|
| neutral|Technopolis plans...|   190|
|negative|The international...|   228|
|positive|With the new prod...|   206|
|positive|According to the ...|   203|
|positive|FINANCING OF ASPO...|   178|
|positive|For the last quar...|   193|
|positive|In the third quar...|   125|
|positive|Operating profit ...|   122|
|positive|Operating profit ...|   103|
|positive|TeliaSonera TLSN ...|   178|
|positive|STORA ENSO , NORS...|   164|
|positive|A purchase agreem...|   207|
|positive|Finnish Talentum ...|   163|
|positive|Clothing retail c...|   143|
|positive|Consolidated net ...|   163|
|positive|Foundries divisio...|   254|
|positive|HELSINKI ( AFX ) ...|   237|
|positive|Incap Contract Ma...|   138|
|positive|Its board of dire...|   125|
+--------+--------------------+------+
only showing top 20 rows



In [455]:
df2 = data_df.withColumnRenamed("_c0", "class").withColumnRenamed("_c1", "text")
df2.printSchema()
df2.show()


root
 |-- class: string (nullable = true)
 |-- text: string (nullable = true)
 |-- length: integer (nullable = true)

+--------+--------------------+------+
|   class|                text|length|
+--------+--------------------+------+
| neutral|According to Gran...|   127|
| neutral|Technopolis plans...|   190|
|negative|The international...|   228|
|positive|With the new prod...|   206|
|positive|According to the ...|   203|
|positive|FINANCING OF ASPO...|   178|
|positive|For the last quar...|   193|
|positive|In the third quar...|   125|
|positive|Operating profit ...|   122|
|positive|Operating profit ...|   103|
|positive|TeliaSonera TLSN ...|   178|
|positive|STORA ENSO , NORS...|   164|
|positive|A purchase agreem...|   207|
|positive|Finnish Talentum ...|   163|
|positive|Clothing retail c...|   143|
|positive|Consolidated net ...|   163|
|positive|Foundries divisio...|   254|
|positive|HELSINKI ( AFX ) ...|   237|
|positive|Incap Contract Ma...|   138|
|positive|Its board of d

In [456]:
# Tokenize DataFrame
tokenizer = Tokenizer(inputCol="text", outputCol="Words")

In [457]:
# Transform DataFrame
reviewed = tokenizer.transform(df2)
reviewed.show()

+--------+--------------------+------+--------------------+
|   class|                text|length|               Words|
+--------+--------------------+------+--------------------+
| neutral|According to Gran...|   127|[according, to, g...|
| neutral|Technopolis plans...|   190|[technopolis, pla...|
|negative|The international...|   228|[the, internation...|
|positive|With the new prod...|   206|[with, the, new, ...|
|positive|According to the ...|   203|[according, to, t...|
|positive|FINANCING OF ASPO...|   178|[financing, of, a...|
|positive|For the last quar...|   193|[for, the, last, ...|
|positive|In the third quar...|   125|[in, the, third, ...|
|positive|Operating profit ...|   122|[operating, profi...|
|positive|Operating profit ...|   103|[operating, profi...|
|positive|TeliaSonera TLSN ...|   178|[teliasonera, tls...|
|positive|STORA ENSO , NORS...|   164|[stora, enso, ,, ...|
|positive|A purchase agreem...|   207|[a, purchase, agr...|
|positive|Finnish Talentum ...|   163|[f

In [458]:
# Remove stop words
# stop_list = ["-","``"]
remover = StopWordsRemover(inputCol="Words", outputCol="filtered", stopWords=stop_list)

In [459]:
# Transform new DataFrame
newFrame = remover.transform(reviewed)
newFrame.show()

+--------+--------------------+------+--------------------+--------------------+
|   class|                text|length|               Words|            filtered|
+--------+--------------------+------+--------------------+--------------------+
| neutral|According to Gran...|   127|[according, to, g...|[gran, ,, company...|
| neutral|Technopolis plans...|   190|[technopolis, pla...|[technopolis, pla...|
|negative|The international...|   228|[the, internation...|[international, e...|
|positive|With the new prod...|   206|[with, the, new, ...|[production, plan...|
|positive|According to the ...|   203|[according, to, t...|[company, 's, upd...|
|positive|FINANCING OF ASPO...|   178|[financing, of, a...|[financing, aspoc...|
|positive|For the last quar...|   193|[for, the, last, ...|[quarter, 2010, ,...|
|positive|In the third quar...|   125|[in, the, third, ...|[quarter, 2010, ,...|
|positive|Operating profit ...|   122|[operating, profi...|[operating, profi...|
|positive|Operating profit .

In [460]:
filtered_df = newFrame.select('filtered')
# filtered_df.select("filtered").write.save("filtered.json",format="json")

In [461]:
# Create all the features to the data set
pos_neg_to_num = StringIndexer(inputCol='class',outputCol='label')
# tokenizer = Tokenizer(inputCol="text", outputCol="token_text")
# stopremove = StopWordsRemover(inputCol='token_text',outputCol='stop_tokens')
hashingTF = HashingTF(inputCol="filtered", outputCol='hash_token')
idf = IDF(inputCol='hash_token', outputCol='idf_token')


In [462]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import Vector

# Create feature vectors
clean_up = VectorAssembler(inputCols=['idf_token', 'length'], outputCol='features')

In [463]:
# Create a and run a data processing Pipeline
from pyspark.ml import Pipeline
data_prep_pipeline = Pipeline(stages=[pos_neg_to_num, tokenizer, remover, hashingTF, idf, clean_up])

In [464]:
# Fit and transform the pipeline
cleaner = data_prep_pipeline.fit(df2)
cleaned = cleaner.transform(df2)

In [465]:
# Show label and resulting features
cleaned.select(['label', 'features']).show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(262145,[32817,45...|
|  0.0|(262145,[20102,21...|
|  2.0|(262145,[4333,939...|
|  1.0|(262145,[29917,32...|
|  1.0|(262145,[3968,440...|
|  1.0|(262145,[3968,811...|
|  1.0|(262145,[45404,47...|
|  1.0|(262145,[33998,38...|
|  1.0|(262145,[33998,38...|
|  1.0|(262145,[33998,38...|
|  1.0|(262145,[35529,45...|
|  1.0|(262145,[20131,32...|
|  1.0|(262145,[4978,392...|
|  1.0|(262145,[33998,38...|
|  1.0|(262145,[8254,166...|
|  1.0|(262145,[11972,22...|
|  1.0|(262145,[9144,306...|
|  1.0|(262145,[2114,688...|
|  1.0|(262145,[4371,246...|
|  1.0|(262145,[21654,45...|
+-----+--------------------+
only showing top 20 rows



In [466]:
from pyspark.ml.classification import NaiveBayes
# Break data down into a training set and a testing set
training, testing = cleaned.randomSplit([0.7, 0.3])

# Create a Naive Bayes model and fit training data
nb = NaiveBayes()
predictor = nb.fit(training)

In [467]:
 # Tranform the model with the testing data
test_results = predictor.transform(testing)
test_results.show(truncate=False)

+--------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------+-----+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------

In [468]:
# Use the Class Evaluator for a cleaner description
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

acc_eval = MulticlassClassificationEvaluator()
acc = acc_eval.evaluate(test_results)
print("Accuracy of model at predicting reviews was: %f" % acc)

Accuracy of model at predicting reviews was: 0.542179


Even with building on stopWord list, we find that performance is not considerably improved.